In [1]:
from utils.binaries import *
from utils.plotting import *

16:24:24 (   +1.6s) [INFO   ] -- import logging
16:24:24 (  +230ms) [INFO   ] -- import numpy as np
16:24:24 (   +13ms) [INFO   ] -- import uncertainties
16:24:25 (  +863ms) [INFO   ] -- import pandas as pd
16:24:25 (    +1ms) [WARNING] -- pandas support to be dropped at some point!
16:24:25 (    +2ms) [INFO   ] -- import binaries.tools as tools
16:24:26 (  +550ms) [INFO   ] -- import matplotlib.pyplot as plt
16:24:27 (   +1.1s) [INFO   ] -- import seaborn as so
16:24:27 (    +6ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
16:24:27 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
16:24:27 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
16:24:27 (    +1ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
16:24:27 (    +0ms) [INFO   ] -- set SCAN_PATH = PosixPath('/cr/data01/filip/xy-calibration')
16:24:27 (    +1ms) [INFO   ] -- set OFLN_PATH = PosixPath('/cr/data01/filip/offline/insta

In [8]:
def read_file(run, station):

    path = f"/cr/data01/filip/Data/muonAcquisitionIII/data/{run}/muonAcqStore.out_{station}.out"
    all_hist = lambda x: [int(e.split(" ")[-1]) for e in x]
    coinc_hist = lambda x: [int(e.split(" ")[-4]) for e in x]  # ...
    peak_histo, rates = [], []

    with open(path, "r") as source:

        baseline = 0

        is_peak_histogram = False
        is_all_histogram = False
        for i, line in enumerate(source.readlines()):
            line = line.strip()

            if line.startswith("Offset"):
                baseline = int(line.split(" ")[-1])

            if line == "Peak":
                is_peak_histogram = True
                continue
            elif line == "Charge":
                is_peak_histogram = False

            if is_peak_histogram:
                peak_histo.append(line)
            if line == "======ssd:":
                is_all_histogram = True
                continue

            if is_all_histogram:
                rates.append(line)

        bin0 = int(rates[0].split(" ")[3])

    return bin0 - baseline, np.cumsum(all_hist(rates[1:])[::-1])[::-1] / 600

In [10]:
import os

data = '/cr/data01/filip/Data/muonAcquisitionIII/data'
offsets = {}
int_rates = {}
timestamps = []

for run in os.listdir(data):
    files = os.listdir(f"{data}/{run}")
    if int(run) > 90: continue

    for file in files:
        if file.endswith('.out'):
            if file.startswith('muonAcqStore'):
                station = int(file.split('.')[1][-3:])
                
                try:
                    offset, rate = read_file(run, station)
                except IndexError:
                    continue

                try:
                    offsets[station].append(offset)
                    int_rates[station].append(rate)

                except KeyError:
                    offsets[station] = [offset]
                    int_rates[station] = [rate]
            else: 
                timestamps.append(int(file.split('.')[0]))

tools.pickle_save('/cr/data01/filip/Data/muonAcquisitionIII/time_offset_rates.pkl', [timestamps, offsets, int_rates])